In [1]:
%autosave 0

Autosave disabled


In [ ]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

In [ ]:
!pip install keras-image-helper

In [5]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [6]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [8]:
from keras_image_helper import create_preprocessor

In [9]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [11]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [14]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [16]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [18]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [22]:
preds = pb_response.outputs['dense_7'].float_val

In [23]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [24]:
dict(zip(classes, preds))

{'dress': -1.8682901859283447,
 'hat': -4.761244773864746,
 'longsleeve': -2.316983461380005,
 'outwear': -1.062570333480835,
 'pants': 9.88715934753418,
 'shirt': -2.8124334812164307,
 'shoes': -3.666282892227173,
 'shorts': 3.200361490249634,
 'skirt': -2.6023383140563965,
 't-shirt': -4.835045337677002}